In [1]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

# Django ORM CRUD Operations Tutorial

CRUD stands for Create, Read, Update, and Delete - the four basic operations for database management systems. In this tutorial, we'll explore how to perform these operations using Django ORM with the NewsData model.

## NewsData Model Structure

Let's first examine the structure of our NewsData model to understand its fields.

In [2]:
# Inspect the NewsData model fields
print('NewsData Model Fields:')
for field in NewsData._meta.get_fields():
    print(f'- {field.name}: {field.get_internal_type()}')

# Display total count of news entries
print(f'\nTotal news entries in database: {NewsData.objects.count()}')

NewsData Model Fields:
- item_id: CharField
- date: DateField
- category: CharField
- title: TextField
- content: TextField
- sentiment: FloatField
- top_key_freq: TextField
- tokens: TextField
- tokens_v2: TextField
- entities: TextField
- token_pos: TextField
- link: CharField
- photo_link: CharField

Total news entries in database: 69


## 1. Create (C) - Adding New Data

The 'Create' operation allows us to insert new records into the database. Here are different ways to create new NewsData objects.

In [3]:
# Method 1: Create using the constructor and save() method
new_news1 = NewsData(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news1",
    category="測試類別",
    content="This is a sample content about technology news for demonstration.",
    date=datetime.now().date(),
    photo_link="https://example.com/image1.jpg"
)
# Uncomment to save: new_news1.save()

# Method 2: Using objects.create() method
# This creates and saves in one step
new_news2 = NewsData.objects.create(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news2",
    category="測試類別",
    content="This is sample content about international news for demonstration.",
    date=datetime.now().date(),
    photo_link=None  # Shows that photo_link can be optional
)

# Method 3: Bulk creation (for adding multiple records at once)
news_list = [
    NewsData(
        title=f"Bulk Example {i}", 
        link=f"https://example.com/bulk{i}",
        category="測試類別",
        content=f"Bulk created content {i}",
        date=datetime.now().date()
    ) for i in range(3, 6)
]
# Uncomment to save: NewsData.objects.bulk_create(news_list)

print(f"Created news item: {new_news2.title} (ID: {new_news2.item_id})")
print("Note: The other examples are commented out to prevent adding test data to your database.")

Created news item: 馬斯克蒞臨高科大 (ID: )
Note: The other examples are commented out to prevent adding test data to your database.


In [4]:

# Clean up - delete the example we created
new_news2.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 2. Read (R) - Querying Data

The 'Read' operation retrieves data from the database. Django ORM provides powerful ways to query data.

In [5]:
# Basic querying examples

# 1. Get all records
all_news = NewsData.objects.all()
print(f"Total news items: {all_news.count()}")


Total news items: 69


In [6]:

# 2. Get a specific record by ID
try:
    first_id = NewsData.objects.first().item_id
    specific_news = NewsData.objects.get(item_id=first_id)
    print(f"\nNews by ID {first_id}: {specific_news.title}")
except:
    print("Couldn't retrieve news by ID")



News by ID finance_20250324_8: 股市一直跌誰最慘？「1類人」恐成海嘯第一排　專家曝3方法降低風險


In [8]:

# 3. Filter records
# Get news from a specific category
tech_news = NewsData.objects.filter(category="國際")
print(f"\nTechnology news count: {tech_news.count()}")
if tech_news.exists():
    print(f"Example tech news: {tech_news.first().title}")



Technology news count: 10
Example tech news: 特朗普宣布美國對進口汽車徵收25%關稅


In [10]:

# 4. Complex filtering
# Get news from the last 7 days
one_week_ago = datetime.now().date() - timedelta(days=7)
recent_news = NewsData.objects.filter(date__gte=one_week_ago)
print(f"\nNews from the last 7 days: {recent_news.count()}")



News from the last 7 days: 69


In [11]:

# 5. Ordering
latest_news = NewsData.objects.order_by('-date')[:5]  # Get 5 most recent news
print("\n5 most recent news:")
for news in latest_news:
    print(f"- {news.item_id} {news.date}:{news.category} {news.title}")



5 most recent news:
- world_20250327_1 2025-03-27:國際 特朗普宣布美國對進口汽車徵收25%關稅
- world_20250327_2 2025-03-27:國際 香港長和賣港口 中國商務部專家批「在商不言商」
- world_20250327_3 2025-03-27:國際 川普上任兩個月 各項關稅概況一覽
- world_20250327_4 2025-03-27:國際 中國男大生坐「共享單車兒童椅」　卡死出不來！急求助消防員
- world_20250327_5 2025-03-27:國際 陸2旅客身藏「28隻活體龜」入境…遭海關查獲　離譜走私手法曝


In [12]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title')[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")



Some news titles:
- ('特朗普宣布美國對進口汽車徵收25%關稅',)
- ('香港長和賣港口 中國商務部專家批「在商不言商」',)
- ('川普上任兩個月 各項關稅概況一覽',)


In [30]:
titles_only

<QuerySet [('外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾',), ('幻象飛官獲救 空軍：掌握跳傘時機做出好示範',), ('法製幻象零附件取得不易 空軍：妥善率沒問題',)]>

In [13]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title', flat=True)[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")



Some news titles:
- 特朗普宣布美國對進口汽車徵收25%關稅
- 香港長和賣港口 中國商務部專家批「在商不言商」
- 川普上任兩個月 各項關稅概況一覽


In [14]:

# 7. Text search (simple contains)
keyword = "台灣"
matching_news = NewsData.objects.filter(content__contains=keyword).count()
print(f"\nNews mentioning '{keyword}': {matching_news}")


News mentioning '台灣': 14


In [15]:
from django.db.models import Q, Max, F
from collections import Counter

## 3. Update (U) - Modifying Data

The 'Update' operation modifies existing records in the database. There are several ways to update data in Django.

In [16]:
# Let's demonstrate updates with a temporary test record
test_news = NewsData.objects.create(
    title="Test News for Update Demo",
    link="https://example.com/test-update",
    category="測試",
    content="This is a temporary record to show update operations.",
    date=datetime.now().date()
)


In [17]:

# Method 1: Instance update
print(f"Original title: {test_news.title}")
test_news.title = "Updated Test News Title"
test_news.category = "更新測試"
test_news.save()
print(f"Updated title: {test_news.title}, Category: {test_news.category}")


Original title: Test News for Update Demo
Updated title: Updated Test News Title, Category: 更新測試


In [18]:

# Method 2: QuerySet update (more efficient for multiple records)
# This updates all matching records without loading them into memory
update_count = NewsData.objects.filter(item_id=test_news.item_id).update(
    content="This content was updated using QuerySet.update() method.",
    photo_link="https://example.com/updated-image.jpg"
)
print(f"\nRecords updated via QuerySet: {update_count}")



Records updated via QuerySet: 1


In [19]:
test_news.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 4. Delete (D) - Removing Data

The 'Delete' operation removes records from the database. Django provides various methods to delete data.

In [20]:
# Let's demonstrate updates with a temporary test record
test_news = NewsData.objects.create(
    item_id="item-123",  # This will be auto-generated by the database
    title="Test News for Update Demo",
    link="https://example.com/test-update",
    category="刪除測試",
    content="This is a temporary record to show update operations.",
    date=datetime.now().date()
)


In [21]:
NewsData.objects.get(item_id="item-123").delete()

(1, {'app_user_keyword_db.NewsData': 1})

In [22]:
NewsData.objects.filter( category="刪除測試")

<QuerySet []>

In [23]:
NewsData.objects.filter( category="刪除測試").delete()

(0, {})

# filter_database_fullText

In [24]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        # q_objects會長這樣： Q(content__contains=kw1) | Q(content__contains=kw2) | Q(content__contains=kw3)
        # 這樣就可以用在filter裡面了
        queryset = queryset.filter(q_objects)
    
    return queryset

In [32]:

user_keywords = ['川普', '俄羅斯']  # Example keywords
cond = 'or'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [33]:
for news in queryset:
    print(news.title)

川普上任兩個月 各項關稅概況一覽
法院文件揭密 司法部高層施壓撤銷亞當斯案
餐廳重設戶外餐區 華埠小商家因2原因對生意仍信心不足
躲不過川普「汽車關稅」！新車價格喊漲　驚人數字曝光
收盤／危險啊！權值股倒成一片　大盤跌破2萬2！
股市一直跌誰最慘？「1類人」恐成海嘯第一排　專家曝3方法降低風險


In [34]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [35]:
news_items

<QuerySet [{'category': '國際', 'title': '川普上任兩個月 各項關稅概況一覽', 'link': 'https://tw.news.yahoo.com/%E5%B7%9D%E6%99%AE%E4%B8%8A%E4%BB%BB%E5%85%A9%E5%80%8B%E6%9C%88-%E5%90%84%E9%A0%85%E9%97%9C%E7%A8%85%E6%A6%82%E6%B3%81-%E8%A6%BD-070602268.html', 'photo_link': None}, {'category': '國際', 'title': '法院文件揭密 司法部高層施壓撤銷亞當斯案', 'link': 'https://tw.news.yahoo.com/%E6%B3%95%E9%99%A2%E6%96%87%E4%BB%B6%E6%8F%AD%E5%AF%86-%E5%8F%B8%E6%B3%95%E9%83%A8%E9%AB%98%E5%B1%A4%E6%96%BD%E5%A3%93%E6%92%A4%E9%8A%B7%E4%BA%9E%E7%95%B6%E6%96%AF%E6%A1%88-063116606.html', 'photo_link': None}, {'category': '國際', 'title': '餐廳重設戶外餐區 華埠小商家因2原因對生意仍信心不足', 'link': 'https://tw.news.yahoo.com/%E9%A4%90%E5%BB%B3%E9%87%8D%E8%A8%AD%E6%88%B6%E5%A4%96%E9%A4%90%E5%8D%80-%E8%8F%AF%E5%9F%A0%E5%B0%8F%E5%95%86%E5%AE%B6%E5%9B%A02%E5%8E%9F%E5%9B%A0%E5%B0%8D%E7%94%9F%E6%84%8F%E4%BB%8D%E4%BF%A1%E5%BF%83%E4%B8%8D%E8%B6%B3-062900330.html', 'photo_link': 'https://s.yimg.com/uu/api/res/1.2/sQvI4HVzFa0tBIg6cYZ94A--~B/Zmk9c3RyaW07aD0yMDA7cT04MDt3PTIwMDth

In [36]:
list(queryset.values_list('date', flat=True)) # flat=True to get a flat list 否則會是 tuple

[datetime.date(2025, 3, 27),
 datetime.date(2025, 3, 27),
 datetime.date(2025, 3, 27),
 datetime.date(2025, 3, 27),
 datetime.date(2025, 3, 27),
 datetime.date(2025, 3, 24)]

In [37]:
latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']

In [38]:
latest_date

datetime.date(2025, 3, 27)

In [39]:
from django.db.models import Avg, Count, Min, Max, Sum
from django.db.models.functions import Extract

# Aggregate是用來計算總和、平均值、最小值、最大值等統計數據的函數
# 這些函數可以用來對查詢集進行聚合計算，並返回一個字典，其中包含計算結果的鍵值對
# Basic aggregation examples
aggregations = NewsData.objects.aggregate(
    count=Count('item_id'),
    oldest=Min('date'),
    newest=Max('date'),
)
print("Basic aggregations:")
for key, value in aggregations.items():
    print(f"- {key}: {value}")


Basic aggregations:
- count: 69
- oldest: 2025-03-24
- newest: 2025-03-27


In [40]:

# Category-based aggregation
category_counts = NewsData.objects.values('category').annotate(
    count=Count('item_id')
).order_by('-count')[:5]  # Top 5 categories
print("\nTop 5 categories by count:")
for item in category_counts:
    print(f"- {item['category']}: {item['count']} articles")


Top 5 categories by count:
- 遊戲3C: 10 articles
- 財經: 10 articles
- 社會地方: 10 articles
- 政治: 10 articles
- 國際: 10 articles


In [41]:
# annotate是用來在查詢集中添加計算字段的函數
# 這些計算字段可以用來進行進一步的過濾、排序或分組操作
# values是用來選擇查詢集中的特定字段的函數 字段就是資料庫中的欄位
# Date-based aggregation (articles per day)
date_counts = NewsData.objects.values('date').annotate(
    count=Count('item_id')
).order_by('-date')[:7]  # Last 7 days with data

print("\nArticles per day (most recent 7 days with data):")
for item in date_counts:
    print(f"- {item['date']}: {item['count']} articles")



Articles per day (most recent 7 days with data):
- 2025-03-27: 55 articles
- 2025-03-26: 7 articles
- 2025-03-25: 4 articles
- 2025-03-24: 3 articles
